<a href="https://colab.research.google.com/github/brianadit24/PySpark-Try/blob/main/Part_6_PySpark_MLib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/krishnaik06/Pyspark-With-Python.git

Cloning into 'Pyspark-With-Python'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 34 (delta 15), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [5]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 73 kB/s 
     |████████████████████████████████| 198 kB 43.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=ceafa4cae818b6df2beaf62ac2006d9fdfd0ec04f1749187ca550161151780f7
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


## Example Of PySpark ML

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Training').getOrCreate()

In [9]:
## Read Dataset
df = spark.read.csv('test1.csv', header=True, inferSchema=True)
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [10]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [11]:
df.columns

['Name', 'age', 'Experience', 'Salary']

[Age, Experience]----> New Feature ----> Independent Feature

In [13]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['age', 'Experience'],
                            outputCol="Independent Features")

In [14]:
output = featureassembler.transform(df)

In [15]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [16]:
finalized_data = output.select("Independent Features", "Salary")
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [18]:
from pyspark.ml.regression import LinearRegression

## Train Test Split
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent Features',
                             labelCol='Salary')
regressor = regressor.fit(train_data)

In [19]:
## Coefficients
regressor.coefficients

DenseVector([-64.8464, 1584.7554])

In [20]:
## Intercepts
regressor.intercept

15414.10693970376

In [24]:
## Prediction
pred = regressor.evaluate(test_data)
pred.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [24.0,3.0]| 20000|18612.059158134223|
+--------------------+------+------------------+



In [25]:
## Scoring
pred.meanAbsoluteError, pred.meanSquaredError, pred.rootMeanSquaredError

(1387.9408418657767, 1926379.780519081, 1387.9408418657767)